# Comparison against First Best

The goal here is to compare both the traditional and mechanism-implemented game against the first best option.

In [1]:
import os
import glob
import numpy as np
import pandas as pd
import math
import random
import itertools
from typing import List, Dict
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import seaborn as sns
from collections import namedtuple, defaultdict
from pandas.plotting import parallel_coordinates
# from PSSimPy.simulator import ABMSim
from PSSimPy import Bank, Account
from PSSimPy.credit_facilities import AbstractCreditFacility
from PSSimPy.utils import add_minutes_to_time, minutes_between

from MDP import MechMDPSearch, MDPStateExt, TradMDPSearch, MDPStateTrad
from SimClasses import ABMSim
from FirstBest import TransactionPath, FirstBestSearch, generate_action_sets

In [6]:
def calculate_delay_cost(sim_name, account: str, delta: float, delta_prime: float):
    df_processed_txns = pd.read_csv(f"{sim_name}-processed_transactions.csv")

    # Filter transactions for the given account
    relevant_txns = df_processed_txns[df_processed_txns['from_account'] == account]
    
    # Initialize total delay cost
    total_delay_cost = 0
    
    # Calculate delay cost for each transaction
    for _, row in relevant_txns.iterrows():
        # Extract submission_time and time as datetime objects
        submission_time = pd.to_datetime(row['submission_time'])
        txn_time = pd.to_datetime(row['time'])
        
        # Calculate the delay in minutes (convert Timedelta to minutes)
        delay_minutes = (submission_time - txn_time).total_seconds() / 60
        
        # Calculate the delay cost
        delay_cost = (delay_minutes / 15) * (delta + delta_prime)
        
        # Accumulate the delay cost
        total_delay_cost += delay_cost
    
    return total_delay_cost

In [7]:
def calculate_borrowing_cost(sim_name, account: str):
    df_credit_facility = pd.read_csv(f'{sim_name}-credit_facility.csv')
    
    # Filter for the given account
    relevant_credit = df_credit_facility[df_credit_facility['account'] == account]

    # Initialize total borrowing cost
    total_borrowing_cost = 0

    # Calculate borrowing cost at each period
    for _, row in relevant_credit.iterrows():
        borrowing_cost = row['total_fee']
        total_borrowing_cost += borrowing_cost
    
    return total_borrowing_cost

## Traditional vs First Best

## Mechanism vs First Best

### Setup

In [2]:
class CollateralizedCreditFacility(AbstractCreditFacility):
    def __init__(self, collateralized_transactions=None, gamma=0.6, phi=0.5, chi=0.75):
        AbstractCreditFacility.__init__(self)
        self.collateralized_transactions = collateralized_transactions if collateralized_transactions is not None else {}
        self.gamma = gamma  # traditional borrowing cost
        self.phi = phi      # pledged-collateral borrowing cost
        self.chi = chi      # unsecured borrowing cost
        # Track credit issued: account.id -> list of tuples (credit_type, amount)
        self.used_credit = {}
        self.history = {}

    def lend_credit(self, account, amount: float) -> None:
        """
        Issue credit to *account* for *amount* according to the rules below.
        ──────────────────────────────────────────────────────────────────────────
        γ  : marginal cost of credit secured by posted collateral
        φ  : marginal cost of credit secured by incoming transactions
        χ  : marginal cost of unsecured credit

        1. If χ is the lowest of {γ, φ, χ}: lend the full *amount* unsecured.

        2. If account.posted_collateral ≥ amount (i.e. collateral is sufficient):
        2a. If φ < χ, first exhaust every valid incoming transaction (status_code
            == 0 and not yet pledged) as collateral, looping until either no
            valid txns remain or credit need is met.
        2b. Use posted collateral for any remainder (or the whole amount if
            φ ≥ χ).

        3. If account.posted_collateral < amount (i.e. collateral is insufficient):
        3a. If χ < φ, lend the full *amount* unsecured.
        3b. Else, first exhaust valid incoming transactions as collateral.
            If credit is still needed, lend the balance unsecured.

        Every tranche advanced is logged in
        self.used_credit[account.id]  →  List[Tuple[str, float, *extra*]]
        so later cost calculations can distinguish ‘unsecured’,
        ‘collateralized_txn’, and ‘collateralized_posted’.
        ──────────────────────────────────────────────────────────────────────────
        """

        # ────────────────────── internal helpers ──────────────────────────
        def _record(kind: str, qty: float, ref=None):
            self.used_credit.setdefault(account.id, []).append((kind, qty, ref))

        def _issue_unsecured(qty: float) -> float:
            if qty > 0:
                account.balance += qty
                _record("unsecured", qty)
            return 0.0

        def _use_posted_collateral(qty: float) -> float:
            take = min(qty, account.posted_collateral)
            if take > 0:
                account.posted_collateral -= take
                account.balance += take
                _record("collateralized_posted", take)
            return qty - take

        def _use_incoming_txn_collateral(qty: float) -> float:
            if qty <= 0:
                return 0.0
            # all un‑pledged, settled‑status incoming txns, largest first
            valid = [
                t for t in account.txn_in
                if t.status_code == 0
                and t not in self.collateralized_transactions.get(account.id, set())
            ]
            valid.sort(key=lambda t: t.amount, reverse=True)

            for txn in valid:
                if qty == 0:
                    break
                take = min(qty, txn.amount)
                self.collateralized_transactions.setdefault(account.id, set()).add(txn)
                account.balance += take
                _record("collateralized_txn", take, txn)
                qty -= take
            return qty
        # ────────────────── end helpers ───────────────────────────────

        remaining = amount

        # Rule 1 ─ unsecured is globally cheapest
        if self.chi < self.gamma and self.chi < self.phi:
            _issue_unsecured(remaining)
            return

        # Rule 2 ─ posted collateral is at least as large as the need
        if account.posted_collateral >= remaining:
            if self.phi < self.chi:
                remaining = _use_incoming_txn_collateral(remaining)
            remaining = _use_posted_collateral(remaining)
            # The branch guarantees sufficiency, but belt‑and‑braces:
            if remaining:
                _issue_unsecured(remaining)
            return

        # Rule 3 ─ posted collateral is insufficient
        if self.chi < self.phi:
            _issue_unsecured(remaining)
            return

        # Prefer incoming‑txn collateral first, then unsecured for any leftover
        remaining = _use_incoming_txn_collateral(remaining)
        _issue_unsecured(remaining)

    def collect_all_repayment(self, day: int, accounts: List[Account]) -> None:
        """
        Collect repayments from all accounts.
        """
        for account in accounts:
            self.history.setdefault(account.id, []).append(
                (day, self.get_total_credit(account), self.get_total_fee(account))
            )
            self.collect_repayment(account)

    def calculate_fee(self, credit_amount, credit_type) -> float:
        """
        Calculate the fee for a given credit amount based on its type.
        
        - For 'collateralized_posted', fee = gamma * credit_amount.
        - For 'collateralized_txn', fee = phi * credit_amount.
        - For 'unsecured', fee = chi * credit_amount.
        """
        if credit_type == 'collateralized_posted':
            return self.gamma * credit_amount
        elif credit_type == 'collateralized_txn':
            return self.phi * credit_amount
        elif credit_type == 'unsecured':
            return self.chi * credit_amount
        else:
            return 0.0


    def collect_repayment(self, account: Account) -> None:
        """
        Use account.balance to repay outstanding tranches in descending
        cost order (largest of gamma, phi, chi first).
        """
        bal = account.balance
        if bal <= 0:
            return

        # 1) Build dynamic cost map
        cost_map = {
            'collateralized_posted': self.gamma,
            'collateralized_txn':    self.phi,
            'unsecured':             self.chi
        }
        # 2) Sort credit types by descending cost
        repay_order = sorted(cost_map, key=lambda k: cost_map[k], reverse=True)

        # 3) Group existing used_credit entries by their kind
        entries = self.used_credit.get(account.id, [])
        by_kind = defaultdict(list)
        for entry in entries:
            kind = entry[0]
            by_kind[kind].append(entry)

        new_entries = []
        # 4) Repay for each kind in that order
        for kind in repay_order:
            for entry in by_kind.get(kind, []):
                if bal <= 0:
                    # no cash left, carry over the entire tranche
                    new_entries.append(entry)
                    continue

                # Unpack amount and optional ref
                amt = entry[1]
                ref = entry[2] if len(entry) > 2 else None

                # Repay as much as we can
                repay_amt = min(amt, bal)
                bal -= repay_amt

                # If posted-collateral, return collateral
                if kind == 'collateralized_posted':
                    account.posted_collateral += repay_amt

                # If txn-collateral, free up txn on full repayment
                elif kind == 'collateralized_txn' and ref is not None:
                    if repay_amt == amt:
                        self.collateralized_transactions[account.id].discard(ref)

                # Unsecured has no extra bookkeeping

                # If partially repaid, keep remainder for next day
                rem = amt - repay_amt
                if rem > 0:
                    if ref is not None:
                        new_entries.append((kind, rem, ref))
                    else:
                        new_entries.append((kind, rem))

        # 5) Commit updates
        account.balance = bal
        self.used_credit[account.id] = new_entries

    def get_total_credit(self, account: Account) -> float:
        """
        Return the aggregate amount of credit issued to *account*,
        independent of credit source.  Works even if each log entry
        carries extra metadata.
        """
        return sum(entry[1] for entry in self.used_credit.get(account.id, []))

    def get_total_fee(self, account: Account) -> float:
        """
        Sum all fees owed by *account* based on the credit tranches logged
        in self.used_credit.  Works whether each log entry is
        (ctype, amount)
        or
        (ctype, amount, extra_metadata).
        """
        total_fee = 0.0
        for entry in self.used_credit.get(account.id, []):
            credit_type = entry[0]      # always first element
            amount      = entry[1]      # always second element
            total_fee  += self.calculate_fee(
                credit_amount=amount,
                credit_type=credit_type
            )
        return total_fee

    
    def get_total_credit_and_fee(self, account: Account) -> float:
        """
        Obtain the total credit and fee for an account.
        """
        return self.get_total_credit(account) + self.get_total_fee(account)

In [ ]:
action_sets = generate_action_sets(10)

In [10]:
def simulate_game(p_t: float, delta: float, gamma: float, delta_prime: float, phi: float, chi: float, seed: int):
    mdp = MechMDPSearch(
        n_players=10,
        n_periods=10,
        p_t=p_t,
        delta=delta,
        delta_prime=delta_prime,
        gamma=gamma,
        phi=phi,
        chi=chi,
        seed=seed
    )

    # Set up strategic bank agent
    class MechStrategicBank(Bank):
        def __init__(self, name, strategy_type='MechStrategic', **kwargs):
            super().__init__(name, strategy_type, **kwargs)
            self.mdp_state = mdp.initial_state() # mdp needs to be redefined before each simulation run
            self.mdp_previous_action = 0
            self.n_periods = 10
        
        # overwrite strategy
        def strategy(self, txns_to_settle: set, all_outstanding_transactions: set, sim_name: str, day: int, current_time: str, queue) -> set:
            if len(txns_to_settle) == 0:
                return set()
            else:
                # we assume 1:1 mapping of bank to account so we can just extract any txn and use that account
                txn = txns_to_settle.copy().pop()
                bank_account = txn.sender_account

            # calculate amount of obligations that arrived in this period
            arrived_obligations = sum([txn.amount for txn in txns_to_settle if txn.arrival_time == current_time])
            # calculate the amount of claims that arrived in this current period
            observed_claims = sum([txn.amount for txn in all_outstanding_transactions if txn.arrival_time == current_time and txn.recipient_account.owner == self.name])

            if current_time == "08:00":
                partial_obs = {
                    "inbound_payments": 0,
                    "arrived_obligations": arrived_obligations,
                    "observed_claims": observed_claims,
                    "observed_expected": 0.75  # not used when ζ = 0
                }

                self.mdp_state = mdp.update_current_state(self.mdp_state, self.mdp_previous_action, partial_obs)
            else:
                # calculate actual inbound payments from previous period
                previous_time = add_minutes_to_time(current_time, -15)
                df_processed_txns = pd.read_csv(f'{sim_name}-processed_transactions.csv')
                filtered_df = df_processed_txns[(df_processed_txns['to_account'] == bank_account) & 
                        (df_processed_txns['settlement_time'] == previous_time)]
                inbound_payments = filtered_df['amount'].sum()

                partial_obs = {
                    "inbound_payments": inbound_payments,
                    "arrived_obligations": arrived_obligations,
                    "observed_claims": observed_claims,
                    "observed_expected": 0.75  # not used when ζ = 0
                }

                self.mdp_state = mdp.update_current_state(self.mdp_state, self.mdp_previous_action, partial_obs)

            _, best_act = mdp.depth_limited_value(self.mdp_state, depth=self.n_periods)
            self.n_periods -= 1
            self.mdp_previous_action = best_act

            if best_act == 1:
                return txns_to_settle
            else:
                return set()

    total_costs = {'mech': 0, 'fb': 0}

    # simulation with mechanism
    strategy_mapping = {'Strategic': MechStrategicBank}
    collateralized_credit_facility = CollateralizedCreditFacility(gamma=gamma, phi=phi, chi=chi)
    banks = pd.read_csv('data_input/bank_template_with_mech_strategy_all10.csv')
    accounts = pd.read_csv('data_input/account_template_mix_collateral.csv')
    mech_sim_name = 'mech_sim'
    mech_sim = ABMSim(name=mech_sim_name, banks=banks, accounts=accounts, strategy_mapping=strategy_mapping, credit_facility=collateralized_credit_facility, open_time='08:00', close_time='10:30', eod_force_settlement=True, txn_arrival_prob=p_t, txn_amount_range=(1,1))
    mech_sim.run()  
    # extract costs
    # delay_costs = {}
    # borrowing_costs = {}
    for account in accounts['id'].tolist():
        # delay_costs[account] = calculate_delay_cost(sim_name, account, delta, delta_prime)
        # borrowing_costs[account] = calculate_borrowing_cost(sim_name, account)
        total_costs['mech'] += (calculate_delay_cost(mech_sim_name, account, delta, delta_prime) + calculate_borrowing_cost(mech_sim_name, account))

    # simulation without mechanism
    transaction_path = TransactionPath()
    df = pd.read_csv(f'{mech_sim_name}-processed_transactions.csv')
    transaction_path.extract_txns_from_df(df)
    accounts_lst = []
    for row in accounts.iterrows():
        account_info = (row[1]['id'], row[1]['balance'], row[1]['posted_collateral'])
        accounts_lst.append(account_info)
    fbs = FirstBestSearch(
        accounts        = accounts_lst,
        delta           = delta,
        gamma           = gamma,
        phi             = phi,
        chi             = chi,
        transaction_path= transaction_path,
        action_sets     = action_sets,
        outstanding_txns= []
    )
    total_costs['fb'] = fbs.run_search()

    # tear down
    # Remove each .csv file
    csv_files = glob.glob("*.csv")
    for file in csv_files:
        os.remove(file)

    return total_costs

### Residual

In [ ]:
# --------- uncontrollable environmental grid (three δ/γ slices) ----------
gamma = 0.3                 # pick a representative collateral cost
delta_ratios = [0.7, 1.0, 1.3]        # mild / knife‑edge / severe delay
p_t = 0.8
seed0 = 1000        # Monte‑Carlo seeds
n_seeds = 1         # small MC replications for stability

# --------- designer‑controlled sweep ----------
phi_vals = np.linspace(0.0, 1.0, 2)    # φ in [0,1]
chi_vals = np.linspace(0.0, 1.0, 2)    # χ in [0,1]

# Fix square aspect ratio and reduce white space
fig, axes = plt.subplots(1, 3, figsize=(15, 5), sharey=True, constrained_layout=True)

for panel, delta_ratio in enumerate(delta_ratios):
    Z = np.zeros((len(phi_vals), len(chi_vals)))
    delta = delta_ratio * gamma

    for i, phi in enumerate(phi_vals):
        for j, chi in enumerate(chi_vals):
            delta_prime = max(0.0, min(max(gamma, phi), chi) - delta)

            fb_costs, mech_costs = [], []
            for k in range(n_seeds):
                res = simulate_game(
                    p_t=p_t,
                    delta=delta,
                    delta_prime=delta_prime,
                    gamma=gamma,
                    phi=phi,
                    chi=chi,
                    seed=seed0 + k
                )
                fb_costs.append(res['fb'])
                mech_costs.append(res['mech'])

            Z[i, j] = np.mean(mech_costs) - np.mean(fb_costs)

    ax = axes[panel]
    im = ax.imshow(
        Z,
        origin='lower',
        extent=[chi_vals[0], chi_vals[-1], phi_vals[0], phi_vals[-1]],
        aspect='equal',  # square!
        cmap='bwr',
        vmin=-np.max(np.abs(Z)),
        vmax=np.max(np.abs(Z))
    )
    ax.set_title(f"δ/γ = {delta_ratio:.1f}")
    ax.set_xlabel("χ  (unsecured cost)")
    if panel == 0:
        ax.set_ylabel("φ  (claim cost)")
    ax.axvline(x=gamma, color='k', ls='--', lw=0.7)
    ax.axhline(y=gamma, color='k', ls=':', lw=0.7)

# Dummy lines for the shared legend
line_chi = mlines.Line2D([], [], color='k', ls='--', lw=0.7, label=r'$\chi=\gamma$')
line_phi = mlines.Line2D([], [], color='k', ls=':',  lw=0.7, label=r'$\phi=\gamma$')

# Shared colorbar with adjusted size
cbar = fig.colorbar(im, ax=axes, shrink=0.8, label="System-cost difference from first-best\n(Mechanism - First-Best)")

# Title
fig.suptitle("Residual welfare gap between the mechanism-implemented game and the first-best benchmark", y=1.02)

plt.show()


In [ ]:
accounts = pd.read_csv('data_input/account_template_mix_collateral.csv')
accounts_lst = []
for row in accounts.iterrows():
    account_info = (row[1]['id'], row[1]['balance'], row[1]['posted_collateral'])
    accounts_lst.append(account_info)

[('acc1', 0, 1000), ('acc2', 0, 1000), ('acc3', 0, 1000), ('acc4', 0, 1000), ('acc5', 0, 1000), ('acc6', 0, 0), ('acc7', 0, 0), ('acc8', 0, 0), ('acc9', 0, 0), ('acc10', 0, 0)]
